## Task 1: Crop faces from the image using face detection results <a name="task1"></a>

We got the results of the face detector model inferece. To continue building our application, we need to use the face detection results to get the face from the full image. Let's try to do it with the first detected face.

### 3. Get the first predicted face from the inferece results

In [49]:
detected_face = detected_faces[0]

### 2. Crop the face from the original image using slices

In [50]:
# Unpack the coordinates from detected_face
xmin = detected_face[0]
ymin = detected_face[1]
xmax = detected_face[2]
ymax = detected_face[3]

# To get the face part of the full image, we will use slice 
# Example, if a face has the coordinates (100,200) (300,400), write: original_image[100:300, 200:400]
face = original_image[ymin:ymax, xmin:xmax]

## Task 2: Classify the face with the mask detector <a name="task2"></a>

The next step is to run the mask detector inference on the face.

### 2. Read the model 
Call core.read_model to read the OpenVINO IR model. The method has two arguments:
    
    1. model - path to the xml model file
    2. weights - path to the bin model file

In [ ]:
mask_detection_model = core.read_model(mask_detection_model_xml, mask_detection_model_bin)

### 4. Compile the model for the device

Use the instance of `Core`.
The class `Core` has a special function called `compile_model`, which compiles a model for a device.
This method prepares the model for inference on the device 
and returns an instance of the model prepared for an execution. 
This function has many parameters, but in this case, you need to know only two of them:
* `model` - instance of `Model`
* `device_name` - string, contains a device name to infer a model on CPU, GPU and other devices.

In [ ]:
mask_detector = core.compile_model(mask_detection_model)

### 6. Define the function to process the image

In [ ]:
def is_masked(face: np.ndarray) -> bool:
    # 1. Infer the model, get predicted probabilities
    mask_detection_results = mask_detector_inference(face)
    
    # 2. The first value - probability of the face with a mask, the second - without a mask
    with_mask_probability = mask_detection_results[0]
    without_mask_probability = mask_detection_results[1]
    
    # 3. compare probs
    has_mask = with_mask_probability > without_mask_probability
    
    return has_mask

## Task 3:  Mark people with and without a mask in the photo <a name="task3"></a>

In this task you need to mark people with a mask in the photo. We prepared the function to process the detected faces. You need to fill out the parts of code to cut a face from the image, put a mark on the face, and paste the processed face image to the original photo.

### 4. Infer the model on the given image

In [ ]:
# Get shape of the original image
original_image_height, original_image_width, _ = original_image.shape

# Infer the face detector
face_detection_inference_result = face_detector_inference(original_image)

# Parse face detection inference results
faces_coordinates = parse_face_detection_results(face_detection_inference_result, original_image_height, original_image_width)

### 5. Process the inference results 

In [ ]:
# Prepare the resulting image
processed_image = original_image.copy()

for face_coordinates in faces_coordinates:
    xmin, xmax, ymin, ymax, _ = face_coordinates
    
    # Get the face from the image
    face = original_image[ymin:ymax, xmin:xmax]

    # Run mask detector 
    is_person_masked = is_masked(face)
    mark = green_mark if is_person_masked else red_cross
        
    # Replace the face with the corresponding mark
    processed_face = put_mark_on_top_of_face(face, mark)
    
    processed_image[ymin:ymax, xmin:xmax] = processed_face

## Task 4: Detect masks on the video <a name="task4"></a>

In [ ]:
# Prepare the resulting image
processed_image = original_image.copy()

for face_coordinates in faces_coordinates:
    xmin = face_coordinates[0]
    ymin = face_coordinates[1]
    xmax = face_coordinates[2]
    ymax = face_coordinates[3]
    
    # To get the face part of the full image, we will use slice 
    # Example, if a face has the coordinates (100,200) (300,400), write: original_image[100:300, 200:400]
    face = original_image[ymin:ymax, xmin:xmax]

    # Run mask detector 
    is_person_masked = is_masked(face)
    mark = green_mark if is_person_masked else red_cross
        
    # Replace the face with the corresponding mark
    processed_face = put_mark_on_top_of_face(face, mark)
    
    # Use the same mechanism as to get the face
    processed_image[ymin:ymax, xmin:xmax] = processed_face